In [1]:
import ParticleFilters: obs_weight
using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPSimulators, QMDP, POMDPPolicies
using POMDPToolbox
using Distributions
using Parameters
using BeliefUpdaters
using Plots
using StaticArrays

┌ Warning: Importing POMDPToolbox. As of Julia 0.7 POMDPToolbox has been discontinued. Please use POMDPModelTools, POMDPPolicies, BeliefUpdaters, and/or POMDPSimulators instead.
└ @ POMDPToolbox C:\Users\cveda\.julia\packages\POMDPToolbox\OdZy7\src\POMDPToolbox.jl:4
┌ Warning: Error requiring `UnicodePlots` from `Plots`
│   exception = (LoadError("C:\\Users\\cveda\\.julia\\packages\\Plots\\CBYO0\\src\\backends\\unicodeplots.jl", 77, UndefVarError(:UserColorType)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000002714a81, Ptr{Nothing} @0x00000000027a282b, Ptr{Nothing} @0x000000000f179de7, Ptr{Nothing} @0x00000000027a1afc, Ptr{Nothing} @0x00000000027a16f4, Ptr{Nothing} @0x00000000027a2048, Ptr{Nothing} @0x00000000027a2c85, Base.InterpreterIP in top-level CodeInfo for Plots at statement 3, Ptr{Nothing} @0x00000000027bf287, Ptr{Nothing} @0x00000000027bfdc1, Ptr{Nothing} @0x00000000027c0d0f, Ptr{Nothing} @0x000000000f754dfb, Ptr{Nothing} @0x000000000f754a1f, Ptr{Nothing} @0x

In [2]:
@with_kw struct RacecarParameters
    # Rewards
    r_crash::Real = -100 #Terminal State
    r_avoided_stop::Real = -4 #Terminal State
    r_avoided_continue::Real = 20 # No Crash yet by continuing straight.
    r_avoided_left::Real = -5 #Terminal State
    r_avoided_right::Real = -6 #Terminal State
    
    # Transition probability
    p_threat_arises::Real = .15   #Probability that the car behind is going to be a threat
    p_crash_when_continue_wthreat::Real = .5
    #Obervation probabilities
    p_collision_detected::Real = .7 # Suboptimal conditions when the lidar only has a 70% chance of detecting the collision
    p_false_alarm::Real = .1        # when no threat
end

RacecarParameters

In [3]:
params = RacecarParameters();

In [4]:
begin
    @enum State Crash Collision_AC Collision_AS Collision_AL Collision_AR 
    @enum Action Continue Stop Left_T Right_T 
    @enum Observation Collision_D Collision_U
end

In [5]:
𝒮 = [Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR]

5-element Vector{State}:
 Crash::State = 0
 Collision_AC::State = 1
 Collision_AS::State = 2
 Collision_AL::State = 3
 Collision_AR::State = 4

In [6]:
𝒜 = [Continue, Stop, Left_T, Right_T]

4-element Vector{Action}:
 Continue::Action = 0
 Stop::Action = 1
 Left_T::Action = 2
 Right_T::Action = 3

In [7]:
𝒪 = [Collision_D, Collision_U]

2-element Vector{Observation}:
 Collision_D::Observation = 0
 Collision_U::Observation = 1

In [8]:
initialstate_distr = Deterministic(Collision_AC)

Deterministic{State}(Collision_AC)

In [9]:
function T(s::State, a::Action)
    p_Crash_I::Real = params.p_crash_when_continue_wthreat
    p_Threat::Real = params.p_threat_arises
    p_Crash::Real = p_Threat*p_Crash_I
    if a == Right_T
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [0,0,0,0,1])
    elseif a == Left_T
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [0,0,0,1,0]) 
    elseif a == Stop
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [0,0,1,0,0])
    elseif a == Continue && s == Collision_AC
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [p_Crash,1-p_Crash,0,0])
    elseif s == Crash
         return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [1,0,0,0,0])
    elseif a == Stop
         return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [0,0,1,0,0])
    else
        return SparseCat([Crash, Collision_AC, Collision_AS, Collision_AL, Collision_AR], [p_Crash,1-p_Crash,0,0])
    end
end

T (generic function with 1 method)

In [10]:
function O(s::State, a::Action, sp::State)
    p_Threat::Real = params.p_threat_arises
    p_True_Detect = params.p_collision_detected
    p_False_Detect = params.p_false_alarm
    p_Normal = p_Threat*p_True_Detect + (1-p_Threat)*p_False_Detect
    if sp == Crash
        return SparseCat([Collision_D, Collision_U], [p_True_Detect, 1-p_True_Detect])
    elseif sp == Collision_AC
        return SparseCat([Collision_D, Collision_U], [p_False_Detect, 1-p_False_Detect])
    else
        return SparseCat([Collision_D, Collision_U], [p_Normal,1-p_Normal])
    end
end        

O (generic function with 1 method)

In [11]:
function R(s::State, a::Action)
    return(s == Crash ? params.r_crash : 0) + (a == Continue ? params.r_avoided_continue : 0) + 
                                              (a == Stop ? params.r_avoided_stop : 0) +
                                              (a == Left_T ? params.r_avoided_left : 0) + 
                                              (a == Right_T ? params.r_avoided_right : 0) 
end

R (generic function with 1 method)

In [12]:
# R(Collision_AC, Continue)

In [13]:
γ = 0.9

0.9

In [14]:
abstract type Racecar <: POMDP{State, Action, Observation}end

In [15]:
pomdp = QuickPOMDP(Racecar,
    states = 𝒮,
    actions = 𝒜,
    observations = 𝒪,
    transition = T,
    reward = R,
    observation = O,
    discount = γ,
    initialstate = initialstate_distr);

In [16]:
struct ActWhenThreat <: Policy end

In [17]:
struct ActWhenBelievedThreat <: Policy end

In [18]:
const Belief = Vector{Real};

In [19]:
function POMDPs.action(::ActWhenThreat, o::Observation)
    return o == Collision_D ? Stop : Continue
end;

In [20]:
function POMDPs.action(::ActWhenBelievedThreat, b::Belief)
    if b[1] > b[2]
        return Stop
    else
        return Continue
    end
end;

In [21]:
updater(pomdp::QuickPOMDP{Racecar}) = DiscreteUpdater(pomdp)

updater (generic function with 1 method)

In [22]:
b0 = uniform_belief(pomdp); b0.b

5-element Vector{Float64}:
 0.2
 0.2
 0.2
 0.2
 0.2

In [23]:
begin 
    a1 = Continue 
    o1 = Collision_U
    b1 = update(DiscreteUpdater(pomdp), b0, a1, o1)
    b1.b
end

5-element Vector{Float64}:
 0.10483870967741937
 0.8951612903225805
 0.0
 0.0
 0.0

In [24]:
using QMDP

In [25]:
solver = QMDPSolver()

QMDPSolver{DiscreteValueIteration.ValueIterationSolver}(DiscreteValueIteration.ValueIterationSolver(100, 0.001, false, true, Float64[]))

In [26]:
policy = solve(solver, pomdp)

AlphaVectorPolicy{QuickPOMDP{Racecar, State, Action, Observation, NamedTuple{(:stateindex, :isterminal, :obsindex, :states, :observations, :discount, :actions, :observation, :actionindex, :transition, :reward, :initialstate), Tuple{Dict{State, Int64}, Bool, Dict{Observation, Int64}, Vector{State}, Vector{Observation}, Float64, Vector{Action}, typeof(O), Dict{Action, Int64}, typeof(T), typeof(R), Deterministic{State}}}}, Action}(QuickPOMDP{Racecar, State, Action, Observation, NamedTuple{(:stateindex, :isterminal, :obsindex, :states, :observations, :discount, :actions, :observation, :actionindex, :transition, :reward, :initialstate), Tuple{Dict{State, Int64}, Bool, Dict{Observation, Int64}, Vector{State}, Vector{Observation}, Float64, Vector{Action}, typeof(O), Dict{Action, Int64}, typeof(T), typeof(R), Deterministic{State}}}}((stateindex = Dict{State, Int64}(Collision_AC => 2, Crash => 1, Collision_AS => 3, Collision_AL => 4, Collision_AR => 5), isterminal = false, obsindex = Dict{Obser

In [27]:
using FIB

In [28]:
fib_solver = FIBSolver()

FIBSolver(100, 0.001, false)

In [29]:
fib_policy = solve(fib_solver, pomdp)

AlphaVectorPolicy{QuickPOMDP{Racecar, State, Action, Observation, NamedTuple{(:stateindex, :isterminal, :obsindex, :states, :observations, :discount, :actions, :observation, :actionindex, :transition, :reward, :initialstate), Tuple{Dict{State, Int64}, Bool, Dict{Observation, Int64}, Vector{State}, Vector{Observation}, Float64, Vector{Action}, typeof(O), Dict{Action, Int64}, typeof(T), typeof(R), Deterministic{State}}}}, Action}(QuickPOMDP{Racecar, State, Action, Observation, NamedTuple{(:stateindex, :isterminal, :obsindex, :states, :observations, :discount, :actions, :observation, :actionindex, :transition, :reward, :initialstate), Tuple{Dict{State, Int64}, Bool, Dict{Observation, Int64}, Vector{State}, Vector{Observation}, Float64, Vector{Action}, typeof(O), Dict{Action, Int64}, typeof(T), typeof(R), Deterministic{State}}}}((stateindex = Dict{State, Int64}(Collision_AC => 2, Crash => 1, Collision_AS => 3, Collision_AL => 4, Collision_AR => 5), isterminal = false, obsindex = Dict{Obser

In [30]:
using Plots; default(fontfamily="Computer Modern", framestyle=:box) # LaTex-style

In [31]:
action(policy, states)

LoadError: MethodError: no method matching support(::typeof(states))
[0mClosest candidates are:
[0m  support([91m::ParticleFilters.ParticleCollection[39m) at C:\Users\cveda\.julia\packages\ParticleFilters\KCmZS\src\beliefs.jl:103
[0m  support([91m::ParticleFilters.AbstractParticleBelief[39m) at C:\Users\cveda\.julia\packages\ParticleFilters\KCmZS\src\beliefs.jl:173
[0m  support([91m::DiscreteBelief[39m) at C:\Users\cveda\.julia\packages\BeliefUpdaters\gFL9O\src\discrete.jl:73
[0m  ...

In [32]:
using BasicPOMCP

In [33]:
pomcp_solver = POMCPSolver()

POMCPSolver
  max_depth: Int64 20
  c: Float64 1.0
  tree_queries: Int64 1000
  max_time: Float64 Inf
  tree_in_info: Bool false
  default_action: ExceptionRethrow ExceptionRethrow()
  rng: Random._GLOBAL_RNG Random._GLOBAL_RNG()
  estimate_value: RolloutEstimator


In [34]:
pomcp_planner = solve(pomcp_solver, pomdp);

In [35]:
initialstate(pomdp)

Deterministic{State}(Collision_AC)

In [36]:
aₚ, info = action_info(pomcp_planner, initialstate(pomdp), tree_in_info=true); aₚ

Right_T::Action = 3

In [37]:
using D3Trees

In [38]:
tree = D3Tree(info[:tree], init_expand=3)

o: <root>
N: 1000      
├──a: Continue
│  N: 2      
│  V: -41       
│  ├──o: Collision_U
│  │  N: 0         
│  │  ├──a: Continue
│  │  │  N: 0      
│  │  │  V: 0          (0 children)
│  │  ├──a: Stop
│  │  │  N: 0      
│  │  │  V: 0          (0 children)
│  │  ├──a: Left_T
│  │  │  N: 0      
│  │  │  V: 0          (0 children)
│  │  └──a: Right_T
│  │     N: 0      
│  │     V: 0          (0 children)
│  └──o: Collision_D
│     N: 0         
│     ├──a: Continue
│     │  N: 0      
│     │  V: 0          (0 children)
│     ├──a: Stop
│     │  N: 0      
│     │  V: 0          (0 children)
│     ├──a: Left_T
│     │  N: 0      
│     │  V: 0          (0 children)
│     └──a: Right_T
│        N: 0      
│        V: 0          (0 children)
├──a: Stop
│  N: 16     
│  V: -17.5     
│  ├──o: Collision_U
│  │  N: 12        
│  │  ├──a: Continue
│  │  │  N: 1      
│  │  │  V: -46.8      (1 children)
│  │  ├──a: Stop
│  │  │  N: 6      
│  │  │  V: -10.9      (2 children)
│  │  ├──a: Left_T
│  │  │  N: 4      
│  │  │  V: -23.3      (2 children)
│  │  └──a: Right_T
│  │     N: 1      
│  │     V: -17.6      (1 children)
│  └──o: Collision_D
│     N: 2         
│     ├──a: Continue
│     │  N: 1      
│     │  V: -14.3      (1 children)
│     ├──a: Stop
│     │  N: 0      
│     │  V: 0          (0 children)
│     ├──a: Left_T
│     │  N: 1      
│     │  V: -29.9      (1 children)
│     └──a: Right_T
│        N: 0      
│        V: 0          (0 children)
├──a: Left_T
│  N: 5      
│  V: -24.8     
│  └──o: Collision_U
│     N: 4         
│     ├──a: Continue
│     │  N: 1      
│     │  V: -137       (1 children)
│     ├──a: Stop
│     │  N: 0      
│     │  V: 0          (0 children)
│     ├──a: Left_T
│     │  N: 1      
│     │  V: 12.2       (1 children)
│     └──a: Right_T
│        N: 2      
│        V: -11.1      (1 children)
└──a: Right_T
   N: 977    
   V: 8.47      
   ├──o: Collision_U
   │  N: 798       
   │  ├──a: Continue
   │  │  N: 4      
   │  │  V: -23.6      (2 children)
   │  ├──a: Stop
   │  │  N: 772    
   │  │  V: 17.9       (2 children)
   │  ├──a: Left_T
   │  │  N: 20     
   │  │  V: -12.7      (2 children)
   │  └──a: Right_T
   │     N: 2      
   │     V: -14.3      (1 children)
   └──o: Collision_D
      N: 177       
      ├──a: Continue
      │  N: 162    
      │  V: 14.8       (2 children)
      ├──a: Stop
      │  N: 2      
      │  V: -9.16      (1 children)
      ├──a: Left_T
      │  N: 7      
      │  V: -9.51      (1 children)
      └──a: Right_T
         N: 6      
         V: -20.4      (2 children)

In [39]:
using POMCPOW

In [40]:
pomcpow_solver = POMCPOWSolver()

POMCPOWSolver{Random._GLOBAL_RNG}
  eps: Float64 0.01
  max_depth: Int64 9223372036854775807
  criterion: MaxUCB
  final_criterion: MaxQ MaxQ()
  tree_queries: Int64 1000
  max_time: Float64 Inf
  rng: Random._GLOBAL_RNG Random._GLOBAL_RNG()
  node_sr_belief_updater: POMCPOW.POWNodeFilter POMCPOW.POWNodeFilter()
  estimate_value: RolloutEstimator
  enable_action_pw: Bool true
  check_repeat_obs: Bool true
  check_repeat_act: Bool true
  tree_in_info: Bool false
  alpha_observation: Float64 0.5
  k_observation: Float64 10.0
  alpha_action: Float64 0.5
  k_action: Float64 10.0
  init_V: Float64 0.0
  init_N: Int64 0
  next_action: MCTS.RandomActionGenerator{Random._GLOBAL_RNG}
  default_action: ExceptionRethrow ExceptionRethrow()


In [41]:
pomcpow_planner = solve(pomcpow_solver, pomdp);

In [42]:
aₚ, info = action_info(pomcpow_planner, initialstate(pomdp), tree_in_info=true); aₚ

Continue::Action = 0

In [43]:
using D3Trees

In [44]:
tree = D3Tree(info[:tree], init_expand=3)

o: <root>
N: 1000      

├──a: Left_T
│  N: 1       V: -18.3     
│  └──o: Collision_U
│     N: 0         

├──a: Stop
│  N: 1       V: -59.3     
│  └──o: Collision_U
│     N: 0         

├──a: Continue
│  N: 994     V: 23.1      
│  ├──o: Collision_U
│  │  N: 877       

│  │  ├──a: Stop
│  │  │  N: 874     V: 11.8       (2 children)
│  │  ├──a: Left_T
│  │  │  N: 1       V: -40.6      (1 children)
│  │  ├──a: Continue
│  │  │  N: 1       V: -56.5      (1 children)
│  │  └──a: Right_T
│  │     N: 1       V: -43.1      (1 children)
│  └──o: Collision_D
│     N: 115       

│     ├──a: Left_T
│     │  N: 4       V: -124       (1 children)
│     ├──a: Continue
│     │  N: 1       V: -168       (1 children)
│     ├──a: Right_T
│     │  N: 1       V: -188       (1 children)
│     └──a: Stop
│        N: 109     V: -53.2      (2 children)
└──a: Right_T
   N: 4       V: -9.16     
   └──o: Collision_U
      N: 3         

      ├──a: Left_T
      │  N: 1       V: -24.6      (1 children)
      └──a: Stop
         N: 2       V: -9.96      (2 children)